In [1]:
!pip install stepic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stepic: filename=stepic-0.5.0-py3-none-any.whl size=12413 sha256=e5a487c6036ef26089a726388e6229fba406817f52d7f80c57faf18040f06921
  Stored in directory: /root/.cache/pip/wheels/0d/e7/18/500b3012b875d302152506f79d026ea3e66c873f7669462d7b
Successfully built stepic


In [32]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.serialization import load_pem_public_key, load_pem_private_key

from PIL import Image
import stepic
import pandas as pd
import hashlib
from google.colab.patches import cv2_imshow as imshow
import matplotlib.pyplot as plt
from google.colab import files
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import secrets

In [33]:
database = pd.DataFrame(columns = ['username', 'password', 'public key'])

database.set_index('username', inplace=True)

In [81]:
def send_secure_email(sender_email, sender_password, recipient_email, subject, body):
    # SMTP server configuration
    smtp_server = "smtp.gmail.com"  # Replace with your SMTP server
    smtp_port = 587  # For TLS (use 465 for SSL)

    # Create the email
    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = recipient_email
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))

    try:
        # Connect to the SMTP server with TLS
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()  # Start TLS encryption
        server.login(sender_email, sender_password)  # Login to the email account
        server.sendmail(sender_email, recipient_email, msg.as_string())  # Send the email
        print("Email sent successfully!")
    except Exception as e:
        print("Error sending email:", e)
    finally:
        server.quit()

In [82]:
def login(username, password):
  try:
    password_hash_object = hashlib.sha256(password.encode())
    password_hash_hex = password_hash_object.hexdigest()


    if username not in database.index:
      print("User not found")
    else:
      if database.loc[username]['password'] == password_hash_hex:

        twofactor = input('Enter [1] for image 2FA. Enter [2] for email 2FA. Lost your key? Enter [3] to generate new key: ')

        if twofactor == '1':
          print("Please upload key image:")
          uploaded = files.upload()

          private_key = extract_key_from_image(list(uploaded.keys())[0])

          os.remove(list(uploaded.keys())[0])


          if private_key == None:
            print('Login Failed.')
            return

          message = (username + password_hash_hex).encode('utf-8')

          if verify_key_pair(private_key, database.loc[username]['public key'], message):
            print("Login Successful!")
          else:
            print("Login Failed.")
        elif twofactor == '2':
          otp = secrets.randbelow(900000) + 100000
          send_secure_email(
          sender_email='issprojectpsut@gmail.com',
          sender_password="iopn vqtj yehl tqni",  # Use an app password for Gmail
          recipient_email=username,
          subject="OTP",
          body="This is your OTP:\n" + str(otp)
          )

          user_otp = input('Enter OTP: ')

          if str(otp) == user_otp:
            print("Login Successful!")
          else:
            print("Login Failed.")
        else:
          otp = secrets.randbelow(900000) + 100000
          send_secure_email(
          sender_email='issprojectpsut@gmail.com',
          sender_password="iopn vqtj yehl tqni",  # Use an app password for Gmail
          recipient_email=username,
          subject="OTP",
          body="This is your OTP:\n" + str(otp)
          )

          user_otp = input('Enter OTP: ')

          if str(otp) == user_otp:

            private_key, public_key = generate_keys()

            database.loc[username, 'public key'] = public_key

            print('Please upload image for key generation:')

            uploaded = files.upload()

            print("Uploaded file:", list(uploaded.keys()))

            embed_key_in_image(list(uploaded.keys())[0], private_key, 'key.png')

            os.remove(list(uploaded.keys())[0])

      else:
        print("Login Failed")
  except:
    print("Login Failed")

def register():
  username = input("Enter username: ")

  if username in database.index:
    print("User already exists.")
    return
  password = input("Enter password:")

  password_hash_object = hashlib.sha256(password.encode())
  password_hash_hex = password_hash_object.hexdigest()

  private_key, public_key = generate_keys()

  database.loc[username] = [password_hash_hex, public_key]

  print('Please upload image for key generation:')

  uploaded = files.upload()

  print("Uploaded file:", list(uploaded.keys()))

  embed_key_in_image(list(uploaded.keys())[0], private_key, 'key.png')

  os.remove(list(uploaded.keys())[0])

In [83]:
def verify_key_pair(private_key_bytes, public_key_bytes, message):

    public_key = load_pem_public_key(public_key_bytes)
    private_key = load_pem_private_key(private_key_bytes, password = None)


    signature = private_key.sign(
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )

    try:
        public_key.verify(
            signature,
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except:
        return False

In [84]:
def generate_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
    )
    public_key = private_key.public_key()

    private_key_bytes = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    public_key_bytes = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    return private_key_bytes, public_key_bytes

In [85]:
def embed_key_in_image(image_path, private_key, output_path):
    with Image.open(image_path) as img:
        encoded_img = stepic.encode(img, private_key)
        encoded_img.save(output_path, format='PNG')

def extract_key_from_image(image_path):
    with Image.open(image_path) as img:
        try:
            private_key = stepic.decode(img)
            return private_key.encode('utf-8')
        except Exception as e:
            return None

In [86]:
def authenticate(username, image_path):

    if True:
      private_key = extract_key_from_image(image_path)

      print("Extracted Private Key:")
      print(private_key.decode('utf-8'))
    else:
      print("Authentication failed!")

In [89]:
if __name__ == "__main__":

  while True:
    choice = input('Enter [1] for Login, and [2] to register a new account. Enter [3] to cancel: ')

    if choice == '1':
      username = input("Enter username: ")
      password = input("Enter password: ")
      login(username, password)
      break
    elif choice == '2':
      register()
      break
    elif choice == '3':
      print('Quitting.')
      break
    else:
      print("Invalid choice. Try again.")

Enter [1] for Login, and [2] to register a new account. Enter [3] to cancel: 1
Enter username: ykarajih@gmail.com
Enter password: password
Enter [1] for image 2FA. Enter [2] for email 2FA. Lost your key? Enter [3] to generate new key: 1
Please upload key image:


Saving key (3).png to key (3).png
Login Successful!
